## Import NBER graph and add department data

In [1]:
import pandas as pd
import networkx as nx
import numpy as np
import scipy.stats as stats
import nltk.metrics as nm
import editdistance
import re
import matplotlib.pyplot as plt
%matplotlib inline

//anaconda/lib/python2.7/site-packages/matplotlib/font_manager.py:273: UserWarning: Matplotlib is building the font cache using fc-list. This may take a moment.
  warnings.warn('Matplotlib is building the font cache using fc-list. This may take a moment.')


## Utility functions

In [2]:
# getJELSubgraph generates the subgraph of G containing all papers with 
# JEL code 'jelcode'
def getJELSubgraph(G, jelcode):
    SG = nx.Graph()
    sge = [e for e in G.edges(data=True) if jelcode in e[2]['jelcode']]
    SG.add_edges_from(sge)
    return SG

In [3]:
# getBigJELSubgraph generates the subgraph of G containing all papers with 
# JEL category (first letter) 'jelcode'
def getBigJELSubgraph(G, jelcode):
    SG = nx.Graph()
    sge = [e for e in G.edges(data=True) if jelcode in re.findall("[a-zA-Z]", e[2]['jelcode'])]
    SG.add_edges_from(sge)
    return SG

In [4]:
# Returns the Mean all-pairs node connectivity of a JEL Code
def getConnEst(G, jelcode):
    return np.mean(nx.all_pairs_node_connectivity(getJELSubgraph(G, jelcode)).values()[0].values()) 

In [5]:
# Get set of JEL super-categories (just first letter)
def getBigJELs(jels):
    bigJels = map(lambda x: re.findall("[a-zA-Z]",x)[0], jels)
    return set(bigJels)

## Begin analysis

In [6]:
# Load NBER Graph from File
# NBER graph contains coauthorship edges annotated with JEL codes 
path = '../save/nber.graphml'
G = nx.read_graphml(path)

In [14]:
# Get List of JEL codes
e = G.edges(data=True)
jels = set(','.join([x[2]['jelcode'] for x in e]).split(','))
bigJels = getBigJELs(jels)

# Get List of Nodes
n = G.nodes()
ndf = pd.DataFrame(n, columns=['author'])

In [9]:
## Merge in Departments (scraped from REPEC)
department = pd.read_csv('../../../work_erica/EconIdeaNetwork/save/REPEC_Paper_Info.csv', delimiter=',')
department = department.drop_duplicates()

In [10]:
department['authorClean'] = department.Author.apply(lambda x: x.replace(',','').split(' '))
department['authorClean'] = department['authorClean'].apply(lambda x: x[1] + x[0])

In [11]:
# HACK: Drop duplicates by taking first institution 
department = department.groupby('authorClean').first().reset_index()

In [12]:
# Manually check for matches using Levenstein Distance (tolerance = 2)

authorDF = []

def getAuthorMatches(author,allAuthors):

    matches = pd.DataFrame(allAuthors.author[allAuthors.author.apply(lambda x: editdistance.eval(x, author)) < 2])
    matches['repecAuthor'] = author

    return matches

In [15]:
# Get list of Author Matches

deptMatches = department.authorClean.apply(lambda x: getAuthorMatches(x,ndf))
deptMatches = pd.concat(list(deptMatches))

In [16]:
# Fix Author Errors in NBER by merging
# Nodes with wrong names

s = deptMatches.groupby('repecAuthor').size()
nberAuthorErrors = s[s > 1].index

for currAuthor in nberAuthorErrors:

    prevAuthorNames = deptMatches[deptMatches.repecAuthor == currAuthor].author.values

    # For each prev name, collect all edges
    # then delete node

    allPrevEdges = []
    allPrevEdgeAttributes = []
    
    for prevName in prevAuthorNames:

        prevEdges = G.edges(prevName, data=True)
        prevEdges = [x[1] for x in prevEdges]
        prevEdgeAttributes = [x[2] for x in prevEdges]

        allPrevEdges = allPrevEdges + prevEdges
        allPrevEdgeAttributes = allPrevEdgeAttributes + prevEdgeAttributes

        G.remove_node(prevName)

    # Create new node with full set of edges and attributes

    for i,target in enumerate(allPrevEdges):
        G.add_edge(prevAuthorNames[0], target, jelcode=allPrevEdgeAttributes[i])

In [20]:
print len(ndf)
print len(G.nodes())

9698
9659


In [ ]:
# Merge Department Affiliations onto Author Matches
authDept = pd.merge(deptMatches, department, how='left', left_on='repecAuthor', right_on='authorClean')

In [ ]:
# Basic full-graph summary stats
degree_dist = nx.degree_histogram(G)

## Summarize a list of subgraphs 

In [ ]:
# utility functions

def getExternalDegreeDist(subgraph, graph):
    degs = np.array(nx.degree(graph, subgraph.nodes()).values()) - \
    np.array(nx.degree(subgraph, subgraph.nodes()).values())
    
    return np.histogram(degs, bins=max(degs))[0]

In [ ]:
def getAvgExternalDegree(subgraph, graph):
    gdegree = np.array(nx.degree(graph, subgraph.nodes()).values())
    sgdegree = np.array(nx.degree(subgraph, subgraph.nodes()).values())
    return np.mean(gdegree - sgdegree)

In [ ]:
# Summarize a set 
# For each graph in a list, compute Avg Degree, Avg Betweenness Centrality, 
# Number of Components,
# Number of Nodes, Longest Shortest Path, Total # of Papers, 
# Avg # of Papers per author, % of Papers within Dept, Rank

# For example, "kind" could be 'Institution', "entries" would be a list of institutions
def summGraphList(kind, entries, subgraphList, graph):
    df = pd.DataFrame()
    df['id']        = entries
    
    df['Subgraph']  = df.id.apply(lambda x: subgraphList[x])
    df['nNodes'] = df.Subgraph.apply(lambda x: len(x.nodes()))
    df['nWithinEdges'] = df.Subgraph.apply(lambda x: len(x.edges()))
    df['nExternalEdges'] = df.Subgraph.apply(lambda x: len(x.edges()))
        
    # degree distribution within the dept
    df['AvgWithinDegree'] = df.Subgraph.apply(lambda x: \
                                         np.mean(nx.degree(x, x.nodes()).values()))

    
    df['WithinDegreeDist'] = df.Subgraph.apply(lambda x: nx.degree_histogram(x))
    df['WithinDegreeSkewness'] = df.WithinDegreeDist.apply(lambda x: stats.skew(x))
    
    # avg degree outside the department
    df['AvgExternalDegree'] = df.Subgraph.apply(lambda x: getAvgExternalDegree(x, graph))
    
    df['ExternalDegreeDist'] = df.Subgraph.apply(lambda x: getExternalDegreeDist(x, graph))
    df['ExternalDegreeSkewness'] = df.ExternalDegreeDist.apply(lambda x: stats.skew(x))
    
    # Drop empty entries
    df = df[df.nNodes != 0]

    # Summary stats for connected components
    df['nConnectedComponents'] = df.Subgraph.apply(lambda x: \
                                   len([c for c in nx.connected_components(x)]))
    df['AvgSizeConnectedComponent'] = df.Subgraph.apply(lambda x: \
                                         np.mean([len(c) for c in nx.connected_components(x)]))
    df['MaxSizeConnectedComponent'] = df.Subgraph.apply(lambda x: \
                                         np.max([len(c) for c in nx.connected_components(x)]))
  
    # betweenness - gonna be very computationally intensive
    # df['BetweennessDist'] = df.Graph.apply(lambda x: nx.betweenness_centrality(x))
    
    # clustering coefficients
    df['clusteringCoeff'] = df.Subgraph.apply(lambda x: average_clustering(x))
    
    # rename id column 
    df.rename(columns={'id': kind})

    # export to latex
    text = df.to_latex()
    fn = sprintf('../save/%s_summtable', kind)
    f = open(fn, 'w')
    return df

## Examine JEL subgraphs

In [ ]:
# save coauthorship network for each JEL category
jelGraphs = {}
overwrite = 1
for currCode in bigJels:
    jelGraphs[currCode] = getBigJELSubgraph(G, currCode)
    if overwrite:
        nx.write_graphml(jelGraphs[currCode], '../save/jel' + currCode + '.graphml')

In [ ]:
# sgl = summGraphList('bigJel', list(bigJels), jelGraphs, G)
jel = 'A4'
[e for e in G.edges(data=True) if jel in re.findall("[a-zA-Z]", e[2]['jelcode'][0])]

In [ ]:
cols = [c for c in sgl.columns if 'id' not in c and 'Subgraph' not in c and 'Dist' not in c]
nCols = len(cols)

fig,ax = plt.subplots(nrows=nCols,ncols=nCols)

for i in range(nCols):
    for j in range(nCols):
        print i,j
        if i == j:
            continue
        else:
            ax[i,j].scatter(sgl[cols[i]], sgl[cols[j]])
            
            

In [ ]:
edd = getExternalDegreeDist(jelGraphs['A'], G)
type(edd[0])

In [ ]:
dd = nx.degree_histogram(G) 

import matplotlib.pyplot as plt
fig, ax = plt.subplots()
ax.bar(np.arange(len(dd)),dd)
plt.show()

In [ ]:
#jeldf = summGraphList('bigJel',bigJels,jelGraphs, G)
jelGraphs

In [ ]:
jelGraphs['A']

## Examine institutional subgraphs 

In [ ]:
# Get list of NBER authors corresponding to Institution

instGraphs = {}
insts = department.Institution.unique()

# save coauthorship network for each institution
for currInst in insts:
    instGraphs[currInst] = G.subgraph(list(authDept[authDept.Institution == currInst].author))
    nx.write_graphml(instGraphs[currInst], \
                     '../save/' + currInst.replace(',','').replace(' ','_') + '.graphml')

In [ ]:
# Summarize Institutional Graphs

instDf = pd.DataFrame()
instDf['Institution'] = insts
instDf['Graph'] = instDf.Institution.apply(lambda x: instGraphs[x])
# Avg Degree, Avg Centrality, Number of Components, Number of Nodes, 
# Longest Shortest Path, Total # of Papers, Avg # of Papers, % of Papers within Dept
# Rank

instDf['AvgDegree'] = instDf.Graph.apply(lambda x: \
                                         np.mean(nx.degree(x, x.nodes()).values()))
instDf['nNodes'] = instDf.Graph.apply(lambda x: len(x.nodes()))

# Drop Empty Departments
instDf = instDf[instDf.nNodes != 0]

instDf['nCC'] = instDf.Graph.apply(lambda x: \
                                   len([c for c in nx.connected_components(x)]))
instDf['AvgSizeCC'] = instDf.Graph.apply(lambda x: \
                                         np.mean([len(c) for c in nx.connected_components(x)]))
instDf['MaxSizeCC'] = instDf.Graph.apply(lambda x: \
                                         np.max([len(c) for c in nx.connected_components(x)]))


In [ ]:
NBERPapers = pd.read_csv('../save/NBER_Paper_Info.1.csv', delimiter='|')

## Netflix feature matrix

- Pagerank
- JEL codes
- Geographic location (state, time zone) (prob time zone)
- Department

In [ ]:
# make a JEL lookup table to map jels to indices                                                     
jelLookup = dict()
for i, jel in enumerate(jels):
    jelLookup[jel] = i

# make an array of authors x JELs 
# to count how many papers in each JEL someone has
authors = nx.nodes(G)
authorCodes = np.zeros((len(authors), len(jels)))
for a in range(len(authors)):
    papers = G.edges(authors[a])
    for p in papers:
        paperAttrs = G.get_edge_data(p[0], p[1])
        paperJels  = paperAttrs['jelcode'].split(',')
        jelInds = [jelLookup[jel] for jel in paperJels]
        authorCodes[a, jelInds] += 1

In [ ]:
# PageRanks
ranks = nx.pagerank(G)

# add to graph                                                                  
nx.set_node_attributes(G, 'rank', ranks)

# in case we want to look at the top authors                                    
sortedRanks = sorted(ranks.items(), key=lambda x: x[1])

In [ ]:
# Extract PageRank to a vector                                                  
authors = G.nodes()
nodeRanks = np.empty(len(authors))
for i in range(len(authors)):
    nodeRanks[i] = ranks.get(authors[i])
nodeRanks = np.reshape(nodeRanks, (len(nodeRanks),1))

In [ ]:
# make feature dataframe
data = np.hstack((nodeRanks,authorCodes))
df = pd.DataFrame(data)
df.columns = ['rank'] + [jel for jel in jels]
df['author'] = authors

In [ ]:
df

In [ ]:
G.nodes(1)

#### TODO 

- Aggregate JELs by letter 
- Add JELS as author attributes.
- Look at degree distribution of subgraphs given by each JEL code